# Development Tools

## Python Virtual Environments

Many Unix/Linux systems come preinstalled with python nowadays. However in most systems the installation of new packages requires superuser privileges. Furthermore the version that is provided by the system may not be the one you want or need. If you have multiple projects, depending on different versions of libraries can be even more cumbersome to maintain.

### Python venv

*More information in the ___[venv docs](https://docs.python.org/3/library/venv.html)___*

For this reason python 3 (since version 3.3) comes with the builtin module **venv** that supports having so called *virtual environments*. This allows a user to install the exact version of the libraries they want to use without needing any superuser privileges. Especially when you work together on projects or use different machines this is a very valuable tool.

To create a new **venv** simply run the command `python -m venv <directory>` in the local directory *<directory>*. To simplify the usage the `venv` comes with an activation script that sets up the local environment for usage of the `venv`. For this bash's builtin `source` command is used to activate the environment. Consecutive `pip install` commands will install the packages into this local directory. See the example below.

In [ ]:
%%bash -e

cd ./tooling/development/trainvenv

# cleanup in case there is already a venv
rm -rf venv || true

# create a new virtual env in folder 'venv' and update pip & setuptools
python -m venv venv --upgrade-deps

# activate 'venv'
source venv/bin/activate

# install all requirements into this environment
pip install -r requirements.txt

echo ">> Printing python version and installed packages of $PWD/venv"
python --version
pip list

The biggest benefit of using this solution is that it already comes bundled with the python standard library (since python 3.3) so it should be available on most systems nowadays.

### Pipenv (and virtualenv)

*More information in the ___[pipenv docs](https://pipenv.pypa.io/en/latest/)___*

`venv` is nice and does its job very well, but if you're coming from the JavaScript/TypeScript world you will most likely want to have a tool such as `npm` to manage your project dependencies.

Python does in fact have an equivalent named **Pipenv**. Pipenv internally uses `pip` and the `virtualenv` package, which is a more advanced virtual environment management tool than `venv` behind the scenes as well as other packages to deliver one whole user experience.

It solves a lot of problems for the user:
- Use one tool to manage the environment instead of multiple (`pip`, `virtualenv`, `Pipfile` ...)
- Hashes are used everywhere for security reasons
- You can get insight into the dependency graph (`pipenv graph`)
- Can automatically load `.env` files

#### Installation

You can either install `pipenv` using your operating systems tools (e.g. `homebrew`, `chocolatey`, `apt`, `dnf` ...) or, since **pipenv** is itself a python tool, by using `pip` itself:

```bash
pip install --user pipenv
```

Omit the `--user` flag if you want to install it system wide.

Attention: Depending on your installation (e.g. homebrew) it may be necessary to manually add the user package directory to the `PATH` environment variable to make use of the `pipenv` command.

#### Pipfile and package management

*pipenv* uses a file called `Pipfile` to describe the environment. Different sections can be defined to shape your virtual environment.

See below an example `Pipfile`:

```Pipfile
[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi"

[packages]
numba = "*"

[dev-packages]
pytest = "*"
faker = "*"
autopep8 = "*"

[requires]
python_version = "3.10"
```

To create a new virtualenv and an empty `Pipfile` enter the project directory and execute the following command to create both by using the currently active python3 version:

In [ ]:
%%bash -e

cd ./tooling/development/pipenv/
# ignore conda env
export PIPENV_IGNORE_VIRTUALENVS=1
# ignore previously set virtual env var
export VIRTUAL_ENV=
# cleanup old stuff
rm -f Pipfile Pipfile.lock requirements.txt
pipenv --rm >/dev/null 2>/dev/null || true

# specify e.g. --python 3.9 if you want to use a specific python version
pipenv install

echo
echo ">>> Resulting Pipfile"
echo
cat Pipfile

Please note that this will also create a file called `Pipfile.lock` to track the exact versions and the dependencies of the installed packages.

If pipenv recognises an already existing `requirements.txt` file in the local directory it will automatically migrate all the packages into the `Pipfile`. If you want to do this manually you can also just use `pipenv install -r requirements.txt`

In [ ]:
%%writefile ./tooling/development/pipenv/requirements.txt
numba

In [ ]:
%%bash -e

cd ./tooling/development/pipenv/
# ignore conda env
export PIPENV_IGNORE_VIRTUALENVS=1
# ignore previously set virtual env var
export VIRTUAL_ENV=
# cleanup old stuff
rm -f Pipfile Pipfile.lock
pipenv --rm >/dev/null 2>/dev/null || true

# install again; now with requirements.txt
pipenv install

echo
echo ">>> Resulting Pipfile"
echo
cat Pipfile

If there is no preexisting `requirements.txt` file, the packages can just be added to the empty `Pipfile` in the `[packages]` section. Another run of `pipenv install` will then install all packages at once.

Alternatively it is also possible to install each package separately by using the syntax `pipenv install <package>`. If pipenv should install a specific version it can be specified by using the corresponding `pip` syntax (e.g. `pipenv install requests~=2.2` or `pipenv install requests>=2.2`).

To find out which packages can be upgraded run `pipenv update --outdated`. Please note that this will only show updates if its allowed by the version specifications (e.g. if you pinned a package to a specific version it won't show new versions, since the version is fixed). To actually run the update use `pipenv update`.

In [ ]:
%%bash -e

cd ./tooling/development/pipenv/
# ignore conda env
export PIPENV_IGNORE_VIRTUALENVS=1
# ignore previously set virtual env var
export VIRTUAL_ENV=

# install an old package and remove the pinning afterwards
pipenv install requests==2.1

echo ">> Old requests version:"
pipenv run pip list | grep "requests "

sed -i "s/==2.1/*/g" Pipfile

# find outdated packages and only list them
pipenv update --outdated || true

# actually update outdated packages
pipenv update

echo ">> Resulting requests version:"
pipenv run pip list | grep "requests "

In order to remove the packages from the virtual environment without altering the `Pipfile` one can run `pipenv uninstall --all` .

#### Enter the environment

To enter the newly built environment run `pipenv shell`. Every command after this will be executed in the subshell and `python` and `pip` executables from the virtual environment will be used.

For example a `pip list` will show the packages installed in the virtual environment. Alternatively also try `pipenv graph` to get a dependency graph of all installed packages.

```bash
$ cd ./tooling/development/pipenv/
$ export PIPENV_IGNORE_VIRTUALENVS=1    # to ignore conda env
$ export VIRTUAL_ENV=                   # reset the virtual env var
$ pipenv shell
(pipenv)$ which python
~/.local/share/virtualenvs/pipenv-Escyejvs/bin/python
(pipenv)$ pip list
Package    Version
---------- -------
mpi4py     3.1.3
pip        22.2.2
setuptools 65.3.0
wheel      0.37.1
```

You can try this out interactively using a terminal launcher in the jupyter notebook and entering the commands above. To leave the subshell just use `exit`.

Instead of opening a shell you can also just execute a command in the environment by using `pipenv run`

In [ ]:
%%bash -e

cd ./tooling/development/pipenv/
# ignore conda env
export PIPENV_IGNORE_VIRTUALENVS=1
# ignore previously set virtual env var
export VIRTUAL_ENV=

pipenv run which python

#### Environment and scripts

With `pipenv` it is also possible to register certain commands for project management as well as set specific environment variables automatically when entering the virtual env or add small scripts that can be run through pipenv.

Run the following code to add a simple script to the Pipfile.

In [ ]:
%%bash -e
if grep "scripts" ./tooling/development/pipenv/Pipfile > /dev/null
then
    exit 0
fi

cat <<EOT >> ./tooling/development/pipenv/Pipfile

[scripts]
echospam = "echo I am really a very silly example"
EOT

The command can now be executed by using its name together with the run command.

In [ ]:
%%bash -e

cd ./tooling/development/pipenv/
# ignore conda env
export PIPENV_IGNORE_VIRTUALENVS=1
# ignore previously set virtual env var
export VIRTUAL_ENV=

pipenv run echospam

In [ ]:
%%bash -e

cd ./tooling/development/pipenv/
# ignore conda env
export PIPENV_IGNORE_VIRTUALENVS=1
# ignore previously set virtual env var
export VIRTUAL_ENV=

# remove the created env to cleanup
pipenv --rm || true

echo "done."

### Poetry

__[Poetry](https://python-poetry.org/)__ is a newer tool for python that is similar in its purpose to `pipenv`. It also provides isolation through virtual environments, manages your package dependencies and provides a lock file to lock down the specific versions. Contrary to `pipenv` it uses the more modern `pyproject.toml` file for storing all relevant project settings and a `poetry.lock` file to lock the the package dependencies.

In addition to this it can also manage `build` and `publish` operations when developing a package that is going to be distributed.

Since this tool wasn't part of the original training material (yet to be updated) we won't go into much more detail at this point but it is a very active project and thus noteworthy. Have a look at it if you are searching for a modern project manager for python.

### Anaconda / Conda

*More information in the __[Conda User Guide](https://docs.conda.io/projects/conda/en/latest/user-guide/index.html)__*

**Anaconda** is a software distribution for python and R provided by Anaconda Inc.

`conda` itself is actually only the package manager and language-agnostic. The tool was deemed necessary by its creators because they identified certain problems with `pip`, the default python package installer, regarding dependency resolution of packages and security issues. It is mainly used to install matching versions of precompiled scientific libraries for python without going through the trouble of compiling them yourself.

If you do not need the full `Anaconda` distribution it is possible to start with the `Miniconda` installer instead and only download the packages that are needed afterwards.

#### Channels

Conda is similar to other package managers and offers different repositories for installing packages. In conda these repositories are called `channels`. By default conda installs packages from the `default` channel (which is Anaconda Inc's own channel).

A very popular conda channel is also the previously mentioned community driven __[conda-forge](https://conda-forge.org/)__ that hosts more recent versions of many packages. There is also another (community) installer named `Miniforge` that has `conda-forge` as main channel per default.

##### Channel priority

`conda` has a highest to lowest channel priority. This means that if a package is found in a higher priority channel (e.g. `default`) and the package satisfies the installation specifications, the packages in the lower priority channels are ignored. If you want to force the install of a version from a lower priority channel you need to use the additional install switch e.g. `--channel conda-forge`.

If you always want to install the newest version available you can also disable the channel priority via `conda config --set channel_priority false`. For more info on channels see __[Managing channels](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-channels.html#)__.

#### Conda init

Since `conda` is itself a python program it utilises bash functions to manage the environment before running any commands. Currently there are three known methods of how to do this init step
- If you want to automatically enable conda when you login to you account you can use `conda init bash`. This command will add the necessary functions to your bash profile configuration file in `~/.bashrc`
- Alternatively this code can be extracted into its own file and sourced whenever `conda` is going to be used (e.g. in a slurm script)
- Another method to install the hooks is by using the following command `eval "$(conda shell.bash hook)"`

Note: Since bash functions are not inherited by subshells we need to call the init functions in each cell before we can (fully) use the conda tool (some commands will work wihtout it). The natural choice for this is the `eval` approach since it always produces the right init code for the specific conda version.

In case you want to have a separate conda init script, you can produce this by running the `conda init bash` command with the `--dry-run --verbose` flags and copy the output into a new shell script.

In [ ]:
%%bash

conda init bash --dry-run --verbose | grep "# >>> conda initialize" -A 100 | grep "# <<< conda initialize" -B 100 | sed 's/+//g' > ./tooling/development/conda/conda-init.sh
cat ./tooling/development/conda/conda-init.sh

#### Environments

`conda` supports having different environments to manage different installations. However it can do a bit more than other virtual environment solutions: with `conda` it is possible to directly install a specific python version for your environment as well as other binary dependencies.

After installing `conda` only the `base` environment is usually available. From there you can create other environments, clone environments, install packages and also export environment settings to a file.

##### Creating a first test environment

To create a simple test environment using the latest available python 3.9 and numba the following statement is enough:

In [ ]:
%%bash

# hook conda functions
eval "$(conda shell.bash hook)"

conda create -y --name test-env --channel conda-forge python=3.9 numba

After conda has solved the constraints and installed all dependencies the environment can be used from the terminal.

Doing the same from e.g. a VSC login node would look like this:

```bash
skylake user@l42:~$ module load miniconda3
skylake user@l42:~$ eval "$(conda shell.bash hook)"
skylake user@l42:~$ conda create -y --name test-env --channel conda-forge python=3.9 numba
... installation ...
(base) skylake user@l42:~$ conda activate test-env
(test-env) skylake user@l42:~$ python3 --version
Python 3.9.16
```

##### Environment Files

To get reproducible environments it is best to pin the specific versions you are currently using (or want to use). Instead of manually keeping track and specifying them in the terminal whenever you want to create a new environment it is better to use conda environment files instead. This file can then be used by yourself or other people to setup the exact same conda environment.

To get a starting point you can simply export the packages from a different active environment (e.g. `base`).

In [ ]:
%%bash

# hook conda functions
eval "$(conda shell.bash hook)"

# activate the base environment since we are in a different one
conda activate base

# export the `base` environment
conda env export --from-history --file ./tooling/development/conda/base-env.yaml

echo 'YAML export fo base conda environment:'
cat ./tooling/development/conda/base-env.yaml

Now lets create our first environment file based on the above output.

For this we will just specify a name, installation channels we want to use and a python version:

In [ ]:
%%writefile ./tooling/development/conda/example-env.yaml
name: example
channels:
  - conda-forge
  - defaults
dependencies:
  - python=3.8

To make use of the environment file we can now use the command `conda env create` and instead of listing each package in the command we specify the environment file.

To customize the installation directory of our new environment we use the `--prefix` flag. If we use a name instead (`--name`) the environment will be installed into the default folder at `~/.conda/envs/<env-name>`.

In [ ]:
%%bash

# hook conda functions
eval "$(conda shell.bash hook)"

conda env create \
    --file ./tooling/development/conda/example-env.yaml \
    --prefix ./tooling/development/conda/example-env

To activate an environment that was created at a specific location we need to use the path instead of the name:

In [ ]:
%%bash

# hook conda functions
eval "$(conda shell.bash hook)"

conda activate ./tooling/development/conda/example-env

which python
python --version
pip list -v

To finally clean up and remove the created environments, run the code below

In [ ]:
%%bash

# hook conda functions
eval "$(conda shell.bash hook)"

# list existing conda environments
conda env list

# remove by name
conda env remove --name test-env

# remove by prefix
conda env remove --prefix ./tooling/development/conda/example-env

# list envs again to see that the test envs were removed
conda env list

## Creating python packages

*More information in the docs at __[Packaging and distributing projects](https://packaging.python.org/en/latest/guides/distributing-packages-using-setuptools/)__ and __[Setuptools](https://setuptools.pypa.io/en/latest/userguide/index.html)__*

### A minimal package (or: the more traditional way)

Creating a basic python package that can be installed with pip from e.g. a repository is fairly easy. All you need for starters is a file called `setup.py` in the root directory of your project. Remember there are lots of other settings so have a look at the __[sampleproject](https://github.com/pypa/sampleproject)__ and the rest of the documentation to get started.


In [ ]:
%%writefile ./tooling/development/minimal-pkg/setup.py

import os

# prefer setuptools over the deprecated distutils
from setuptools import setup, find_packages

username = os.getenv('USER', 'dev')

setup(
    # the name of the package that will be shown everywhere
    name=f'python4hpc-{username}',
    # package version
    version='2023.07.0',
    # optional: author of the package
    author='VSC',
    # optional: author's e-mail
    author_email='service@vsc.ac.at',
    # when the sources are in a subdirectory e.g. `src/` it is necessary to specify this argument
    package_dir={'': 'src'},
    # find_packages searches the 'src' directory for subdirs that contain an '__init__.py' file (=packages) and simply adds them
    packages=find_packages(where='src'),
    # optional: python version requirement e.g. if any features of newer versions are used
    python_requires=">=3.7, <4",
    # optional: setup console scripts
    entry_points={  # Optional
        "console_scripts": [
            "sample=python4hpc.sample:main",
        ],
    },
)

In [ ]:
!pip install ./tooling/development/minimal-pkg

In [ ]:
!pip list | grep python4hpc

In [ ]:
# we need to use the direct path here since the `bin` dir is most likely not on the PATH of the current user
!~/.local/bin/sample

In [ ]:
!pip uninstall --yes python4hpc-$USER

### Packaging and distributing projects

*Taken mostly from __[Packaging Projects](https://packaging.python.org/en/latest/tutorials/packaging-projects/)__*

If you're working on a bigger project and you want to share it with other users its also possible to build a package and upload it to the main python package repository: __[PyPi.org](https://pypi.org)__

The official packaging tutorial suggests to use a `pyproject.toml` to describe the package instead of a `setup.py` and assumes a `src` directory with packages as subdirectories as default. Most of the arguments are very similar to what we have already seen in the case of the `setup.py`.


In [ ]:
%%writefile ./tooling/development/project-pkg/pyproject.toml

[build-system]
requires = ["setuptools>=61.0"]
build-backend = "setuptools.build_meta"

[project]
# don't change this name - will be automatically changed later
name = "python4hpc-pyproject-USERNAME_PLACEHOLDER"
version = "2023.7.0"
authors = [
  { name="Example Author", email="author@example.com" },
]
description = "A small example package"
readme = "README.md"
license = { file="LICENSE" }
requires-python = ">=3.7"
classifiers = [
    "Programming Language :: Python :: 3",
    "License :: OSI Approved :: MIT License",
    "Operating System :: OS Independent",
]

[project.urls]
"Homepage" = "https://github.com/pypa/sampleproject"
"Bug Tracker" = "https://github.com/pypa/sampleproject/issues"


The following command is only used to replace the default package name with a user specific one so that we don't have name clashes on upload.

In [ ]:
%%bash -e
sed -i "s/python4hpc-pyproject-USERNAME_PLACEHOLDER/python4hpc-pyproject-$USER/g" ./tooling/development/project-pkg/pyproject.toml
cat ./tooling/development/project-pkg/pyproject.toml | grep "name ="

Now build the project using the `build` tool from the `build` package - this will create a source distribution and a wheel in the `dist` subdirectory of `/tooling/development/project-pkg`.

In [ ]:
%%bash -e
cd ./tooling/development/project-pkg
python -m build

For our example package we will use the tool **twine** and upload it to `test.pypi.org`. Please create your own account and an access token if you want to run the following snippets (__[Create an account on TestPyPI](https://test.pypi.org/account/register/)__)

In [ ]:
%%writefile ~/.pypirc
[testpypi]
  username = __token__
  password = <INSERT PYPI TOKEN HERE>

In [ ]:
%%bash -e

cd ./tooling/development/project-pkg
python3 -m twine upload --repository testpypi dist/*

In [ ]:
%%bash -e

pip install -i https://test.pypi.org/simple/ python4hpc-pyproject-$USER
echo

echo "List installed packages:"
pip -v list | grep python4hpc-pyproject

In [ ]:
%%bash -e

pip uninstall --yes python4hpc-pyproject-$USER
rm -f ./tooling/development/project-pkg/dist/*

## Apptainer (Singularity)

Singularity is an open source container platform, similar to docker. One of the key differences is, that Singularity favours integration over isolation and comes with out of the box support for graphic accelerators and high-performance interconnects such as e.g. InfiniBand and Omni-Path.

For example in singularity the systems user ids are by default not isolated from the containers user ids. The container instance usually just runs with the users privileges - in addition the home directory is also automatically mounted into the container instance.

In 2021 open source project Singularity was renamed to **Apptainer**. The company Sylabs that provides commercial support as well as a community edition of singularity (SingularityCE) still uses the name **Singularity** for its products.

### Singularity/Apptainer on VSC

Both VSC-4 and VSC-5 come with the singularity packages already installed in the base system.

At the beginning of 2023 the version that is in use is `Singularity 3.8.3`. The documentation for this specific version can be found here: https://apptainer.org/user-docs/3.8/

A slightly newer version `3.8.5` is available in our module system (see `module avail singularity`) and we also provide multiple versions of apptainer (see `module avail apptainer`, currently `1.1.6` and `1.1.9` on VSC-4).

### Build a singularity image from a definition file

There are multiple ways to build a singularity image. If you want to customise an existing docker image or start your own custom image from scratch you can use a so called __definition file__ (`.def`) to describe what should end up in a container image.

Definition files _can_ contain multiple sections such as `%files`, which simply describes files to copy into the container as well as `%test` to run automated tests after the container has been built. Please refer to the latest __[Apptainer documentation](https://apptainer.org/docs/user/latest/)__ for an in-depth description of possible sections and their purpose.

```singularity
Bootstrap: docker
From: ubuntu:22.10

%post
    apt-get -y update
    apt-get -y install cowsay lolcat

%environment
    export LC_ALL=C
    export PATH=/usr/games:$PATH

%runscript
    date | cowsay | lolcat

%test
    grep -q NAME=\"Ubuntu\" /etc/os-release
    if [ $? -eq 0 ]; then
        echo "Container base is Ubuntu as expected."
    else
        echo "Container base is not Ubuntu."
        exit 1
    fi

%help
    This is a demo container used to illustrate a def file that uses all
    supported sections.
```

Once the definition file has been saved you can then build the image by calling

```bash
$ sudo singularity build ./singularity/test.sif ./singularity/test.def
```

This will produce the image file `./singularity/test.sif`

To execute the test script after building the container you can run

```bash
$ sudo singularity test ./singularity/test.sif
```

Unfortunately it is necessary to either have elevated rights (e.g. via `sudo`) or a setup that allows a rootless run (e.g. fakeroot). Since VSC does not provide such an environment (yet) **it is not possible to build a singularity image on our cluster at the moment**.

Thus the current recommendation is to build an image on your local machine and then upload it to VSC.

### Build a singularity image from a docker image

A very common use case is to just take an existing docker image and convert it into the singularity container format. Luckily this can be easily done by executing the following command

```bash
$ singularity build lolcow.sif docker://sylabsio/lolcow
```

<img src="images/development/singularity_build.png" width="500">


In [ ]:
%%bash -e

cd ./tooling/development/singularity
singularity build lolcow.sif docker://sylabsio/lolcow

### Run the image

After the image has been built successfully, you can run a container instance (if it has a runscript) with

```bash
$ singularity run lolcow.sif
```

<img src="images/development/singularity_run.png" width="400">

or execute a command in the container environment with

```bash
$ singularity exec lolcow.sif <command>
```


In [ ]:
%%bash -e

cd ./tooling/development/singularity
singularity run lolcow.sif

In [ ]:
%%bash -e

cd ./tooling/development/singularity
singularity exec lolcow.sif ls -alh

In [ ]:
!rm ./tooling/development/singularity/lolcow.sif

## Development IDE's

### Visual Studio Code (Microsoft)

At its core VS Code is 'just' a simple code editor. However lots of available extensions support the broad variety of use cases of this IDE. *Code* gained immensely in popularity in the past years and has a very active community. It is very well equipped to be used for developing and debugging many programming languages and is available for free on Windows, Mac & Linux (see __[Download Visual Studio Code](https://code.visualstudio.com/download)__).

The recommended way to get started with VS Code & Python Development is to install the *Python* extension (see __[Python in VS Code](https://code.visualstudio.com/docs/languages/python)__) but you can also use e.g. the *Python Extension Pack* by Don Jayamanne which already contains a collection of interesting extensions.

There are also a lot of other extensions to check out e.g.:
* Remote Development Extension Pack - Remote development via e.g. SSH
* Jupyter Extension Pack
* Docker Extension Pack
* Apptainer/Singularity
* Git Extension Pack

and many more ...

With the extensions from the *Python Extension Pack* VS Code recognises python, venv, pipenv and even conda installations and lets the user select the right environment. It is also possible to run python programs interactively or debug them by using the integrated debugger UI, run scripts and manage code through your VCS.

There is also an extensive article about data science usage of VS Code (see __[Data Science in Visual Studio Code](https://code.visualstudio.com/docs/datascience/overview)__).

If you prefer a complete open source community-driven alternative to VS Code check out __[VSCodium](https://vscodium.com/)__

<img src="images/development/vscode_debugging.png" width="550">

### PyCharm (JetBrains)

PyCharm is a commercial product from JetBrains and compared to VS Code a ready to go all-in-one solution for python development. It is based on JetBrains IntelliJ IDEA Platform which was originally conceived for Java Development (see __[PyCharm by JetBrains](https://www.jetbrains.com/pycharm)__)

In the beginning of 2023 it comes in two major editions:
- Community: has all the main features (editor, debugger, refactoring, vcs support) and is free to use as well as open-source
- Professional: has additional features for scientific python, web development and specific python frameworks as well as a profiler & remote development

The IDE is known for its support and integration of tools as well as its refactoring and smart code navigation capabilities and is also available for Windows, Mac & Linux.

There is also an option for educational/classroom licenses for students and academic staff (see __[Educational Licenses](https://www.jetbrains.com/community/education)__) - however it is explicitly stated that it may only be used for non-commercial and training purposes.

Of course PyCharm also comes with its own plugin system and a marketplace, so it can be extended as well.

Especially the builtin Scientific Mode (only available in the professional edition) and the Jupyter notebook support may be interesting because it allows the user to peak interactively into dataframes and also has a special tool for data plots (SciView).

<img src="images/development/pycharm_scientific_mode.jpg" width="550">

see __[Scientific mode tutorial](https://www.jetbrains.com/help/pycharm/matplotlib-tutorial.html#run)__

### Other IDE's

There are also lots of other IDE's to check out that shouldn't go unmentioned as they are either very new or officially referred to in the official python FAQ (see __[Python IDE's for debugging](https://docs.python.org/3.10/faq/programming.html#is-there-a-source-code-level-debugger-with-breakpoints-single-stepping-etc)__)

* __[JupyterLab Desktop](https://github.com/jupyterlab/jupyterlab-desktop)__
* __[Spyder](https://www.spyder-ide.org/)__ (also part of the Anaconda distribution)
* __[Wing Python IDE](https://wingware.com/)__
* __[Komodo IDE](https://www.activestate.com/products/komodo-ide/)__